# Mitigating Bias in a Regression Model

In this notebook, we mitigate bias in a regression model. We will create a pipeline to implement bias mitigation techniques from three levels: Pre-Processing, In-Processing, and Post-Processing. All questions and tasks are bolded and in red.

### 0 - Importing modules and loading the data

We begin by loading the dataset. For this milestone, we will be using the 'USCrime' from the OpenML Repository and loading it directly from the holisticai library. The dataset combines socio-economic data from the 1990 US Census, law enforcement data from the 1990 US LEMAS survey, and crime data from the 1995 FBI UCR.

In [ ]:
# make sure holisticai is installed
!pip install holisticai

In [4]:
# Base Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, max_error
from sklearn.preprocessing import StandardScaler

# import dataset
from holisticai.datasets import load_us_crime

# import plotting functions
from holisticai.bias.plots import histogram_plot
from holisticai.bias.plots import group_pie_plot
from holisticai.bias.plots import distribution_plot
from holisticai.bias.plots import success_rate_curves

# import some bias metrics
from holisticai.bias.metrics import statistical_parity_regression
from holisticai.bias.metrics import disparate_impact_regression
from holisticai.bias.metrics import mae_ratio
from holisticai.bias.metrics import rmse_ratio
from holisticai.bias.metrics import regression_bias_metrics

# import bias mitigation techniques
from holisticai.bias.mitigation import LearningFairRepresentation
from holisticai.bias.mitigation import GridSearchReduction
from holisticai.bias.mitigation import CorrelationRemover
from holisticai.bias.mitigation import ExponentiatedGradientReduction
from holisticai.bias.mitigation import WasserteinBarycenter


# import pipeline function
from holisticai.pipeline import Pipeline

In [5]:
# load the data set
dataset = load_us_crime(return_X_y=False, as_frame=True)
df = pd.concat([dataset["data"], dataset["target"]], axis=1)
df_clean = df.iloc[:,[i for i,n in enumerate(df.isna().sum(axis=0).T.values) if n<1000]]
df_clean = df_clean.dropna()

df_clean

,state,communityname,fold,population,householdsize,racepctblack,racePctWhite,racePctAsian,racePctHisp,agePct12t21,...,PctForeignBorn,PctBornSameState,PctSameHouse85,PctSameCity85,PctSameState85,LandArea,PopDens,PctUsePubTrans,LemasPctOfficDrugUn,ViolentCrimesPerPop
0,8.0,Lakewoodcity,1.0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,...,0.12,0.42,0.50,0.51,0.64,0.12,0.26,0.20,0.32,0.20
1,53.0,Tukwilacity,1.0,0.00,0.16,0.12,0.74,0.45,0.07,0.26,...,0.21,0.50,0.34,0.60,0.52,0.02,0.12,0.45,0.00,0.67
2,24.0,Aberdeentown,1.0,0.00,0.42,0.49,0.56,0.17,0.04,0.39,...,0.14,0.49,0.54,0.67,0.56,0.01,0.21,0.02,0.00,0.43
3,34.0,Willingborotownship,1.0,0.04,0.77,1.00,0.08,0.12,0.10,0.51,...,0.19,0.30,0.73,0.64,0.65,0.02,0.39,0.28,0.00,0.12
4,42.0,Bethlehemtownship,1.0,0.01,0.55,0.02,0.95,0.09,0.05,0.38,...,0.11,0.72,0.64,0.61,0.53,0.04,0.09,0.02,0.00,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1989,12.0,TempleTerracecity,10.0,0.01,0.40,0.10,0.87,0.12,0.16,0.43,...,0.22,0.28,0.34,0.48,0.39,0.01,0.28,0.05,0.00,0.09
1990,6.0,Seasidecity,10.0,0.05,0.96,0.46,0.28,0.83,0.32,0.69,...,0.53,0.25,0.17,0.10,0.00,0.02,0.37,0.20,0.00,0.45
1991,9.0,Waterburytown,10.0,0.16,0.37,0.25,0.69,0.04,0.25,0.35,...,0.25,0.68,0.61,0.79,0.76,0.08,0.32,0.18,0.91,0.23
1992,25.0,Walthamcity,10.0,0.08,0.51,0.06,0.87,0.22,0.10,0.58,...,0.45,0.64,0.54,0.59,0.52,0.03,0.38,0.33,0.22,0.19


### 1- Pre-processing the data

Similar to the bias measurement notebook, we are going to prepare the data for the training of a regression model. We will use sklearn's linear regression model as our model of choice, and we use its train_test_split function to split our dataset. Note, we do not want to include protected attributes in the training so we will remove any features that contain 'race' and 'age' from the features during training and inference.

In [6]:
# choose the attribute we want to measure bias with respect to and assign group membership
gs = ['racePctWhite']
groups = {}
for race in gs:
    groups[race] = df_clean[race].apply(lambda x: x>0.5)

group_a =  groups[gs[0]]
group_b =  1-group_a#groups[gs[1]]
xor_groups  = group_a ^ group_b

# remove sensitive attributes from the training data
cols = [c for c in df_clean.columns if (not c.startswith('race')) and (not c.startswith('age'))]
df_clean = df_clean[cols].iloc[:,3:]
df_clean = df_clean[xor_groups]
group_a = group_a[xor_groups]
group_b = group_b[xor_groups]

# standardize and split the data 
scalar = StandardScaler()
df_t = scalar.fit_transform(df_clean)
X = df_t[:,:-1]
y = df_t[:,-1]

X_train,X_test,y_train,y_test, group_a_tr, group_a_ts, group_b_tr, group_b_ts = \
    train_test_split(X, y, group_a, group_b, test_size=0.2, random_state=42)
train_data = X_train, y_train, group_a_tr, group_b_tr
test_data  = X_test, y_test, group_a_ts, group_b_ts


### 2 - Training the model and measuring bias

<font color='red'>  **Task 1**
- **Train the model using the linear regression function from sklearn. Once trained, caclulate the Statisical Parity, Disparate Impact, MAE Ratio, and RMSE Ratio each at the $q = 0.8$ quantile.**
<font >

In [7]:
# Train a simple linear regression model
LR = LinearRegression()

model = LR.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_true  = np.array(y_test)

# TODO

# TODO

Statistical Parity Q80   : -0.7038208168642952
Disparate Impact Q80     : 0.10067340067340066
MAE Ratio Q80            : 0.80922834612527
RMSE Ratio Q80           : 0.8501595473301025


You should get the following results:

| Metric | Value | Reference |
| --- | --- | --- |
| Statistical Parity Q80 | -0.704 | 0 |
| Disparate Impact Q80   |  0.101 | 1 |
| MAE Ratio Q80          |  0.809 | 1 |
| RMSE Ratio Q80         |  0.850 | 1 |

### 3 - Implementing mitigation techniques

In the following sections, we will implement bias mitigation at three different levels: Pre-Processing, In-Processing, and Post-Processing. To begin, we must create a pipeline to contain the mitigation technique and model itself. First, we will implement a mitigation technique directly, then we will construct a pipeline to hold our model, standardization technique, and mitigation technique.

Let's construct a baseline model to compare the performance of the bias mitigation techniques.

In [8]:
# construct a pipeline
pipeline = Pipeline(
    steps=[
        ('scalar', StandardScaler()),
        ("model", LinearRegression()),
    ]
)

X, y, group_a, group_b = train_data

# fit the training data
pipeline.fit(X, y)


# make prediciton on the test set
X, y, group_a, group_b = test_data

y_pred = pipeline.predict(X)

df = regression_bias_metrics(
    group_a,
    group_b,
    y_pred,
    y,
    metric_type='both'
)
y_baseline = y_pred.copy()
df_baseline=df.copy()
df_baseline



,Value,Reference
Metric,,
Disparate Impact Q90,0.016953,1
Disparate Impact Q80,0.100673,1
Disparate Impact Q50,0.424518,1
Statistical Parity Q50,-0.703821,0
No Disparate Impact Level,-0.825434,-
Average Score Difference,-1.492622,0
Z Score Difference,-2.465747,0
Max Statistical Parity,0.768248,0
Statistical Parity AUC,0.439341,0


#### 3.1 - Pre-Processing Methods for Bias Mitigation

We begin with the Pre-processing techniques. Pre-processing operates on the data level and aims to either combat representation bias or historical bias. To deal with unbalanced data sets experiencing representational bias, pre-processing techniques strategically sample to oversample underrepresented groups. To handle historical bias, data sets that reinforce stereotypes of certain groups, pre-processing techniques aim to remove any proxies to or notions of group membership from the data. We will be implementing the Correlation Remover which applies a linear transformation to the non-sensitive feature columns to remove their correlation with the sensitive feature columns while retaining as much information as possible (as measured by the least-squares error). This method will change the original dataset by removing all correlations with sensitive values. Note that the lack of correlation does not imply anything about statistical dependence. Therefore, it is expected this to be most appropriate as a preprocessing step for (generalized) linear models.

In [12]:
# initialise algorithm
cr = CorrelationRemover()

# standard scale the data
X_train, y_train, group_a, group_b = train_data
scaler1 = StandardScaler()
X_train_t = scaler1.fit_transform(X_train)

# fit the cr to the standardized data
cr.fit(X_train,group_a, group_b)

# transform training data
X_train, y_train, group_a_train, group_b_train = train_data
X_train_t = scaler1.fit_transform(X_train)
new_X_train = cr.transform(X_train_t, group_a_train, group_b_train)

# transform testing data
X_test, y_test, group_a_test, group_b_test = test_data
X_test_t = scaler1.fit_transform(X_test)
new_X_test = cr.transform(X_test_t, group_a_test, group_b_test)

# Fit a model with new data (transformed by cr algorithm)

# train the model
X, y, group_a, group_b = train_data
X = new_X_train
scaler2 = StandardScaler()
Xt = scaler2.fit_transform(X)
model = LinearRegression()
model.fit(Xt, y)

# test the model 
X, y, group_a, group_b = test_data
X = new_X_test
Xt = scaler2.transform(X)
y_pred = model.predict(Xt)

# get metrics
df = regression_bias_metrics(
    group_a,
    group_b,
    y_pred,
    y,
    metric_type='both'
)
y_cr = y_pred.copy()
df_cr = df.copy()
df_cr

,Value,Reference
Metric,,
Disparate Impact Q90,0.209091,1
Disparate Impact Q80,0.219814,1
Disparate Impact Q50,0.499692,1
Statistical Parity Q50,-0.440975,0
No Disparate Impact Level,-0.688300,-
Average Score Difference,-0.961278,0
Z Score Difference,-1.380165,0
Max Statistical Parity,0.522925,0
Statistical Parity AUC,0.304540,0


If we look at the Disparate Impact Q90 and Statistical Parity Q50 metrics, we can see a performance improvement. While there is still a bias against group a, both metrics are closer to their reference value than before.

Now we will implement the same pre-processing technique using the pipeline method. The pipeline allows for a more streamlined approach to training a model and mitigating bias from it. 

In [13]:
# initialize the pipeline 
model = LinearRegression()
pipeline = Pipeline(
    steps=[
        ('scalar', StandardScaler()),
        ("bm_preprocessing", CorrelationRemover()),
        ("model", model),
    ]
)

# prepare training data and parameters
X, y, group_a, group_b = train_data
fit_params = {
    "bm__group_a": group_a, 
    "bm__group_b": group_b
}

# apply steps in pipeline
pipeline.fit(X, y, **fit_params)


# prepare testing data and parameters
X, y, group_a, group_b = test_data
predict_params = {
    "bm__group_a": group_a,
    "bm__group_b": group_b,
}

# make a prediction and generate metrics
y_pred = pipeline.predict(X, **predict_params)
df = regression_bias_metrics(
    group_a,
    group_b,
    y_pred,
    y,
    metric_type='both'
)
y_correm  = y_pred.copy()
df_correm =df.copy()
df_correm

,Value,Reference
Metric,,
Disparate Impact Q90,0.209091,1
Disparate Impact Q80,0.219814,1
Disparate Impact Q50,0.499692,1
Statistical Parity Q50,-0.440975,0
No Disparate Impact Level,-0.763841,-
Average Score Difference,-0.943252,0
Z Score Difference,-1.388035,0
Max Statistical Parity,0.529644,0
Statistical Parity AUC,0.305583,0


Notice the order of the steps in the pipeline. First, the standard scalar is applied, then the pre-processing method and finally the model. Looking at the table of bias metrics, we get the same results as the previous implementation. 

#### 3.2 - In-Processing Methods for Bias Mitigation

Now that we have implemented pre-processing algorithms, the next step is to take a look at one level deeper, in-processing. In-processing algorithms operate at the algorithmic level and are applied *during* training. Many in-processing algorithms add an optimization constraint to the loss function to enforce fairness, while others aim to remove any indicators of sensitive attributes. We will implement the Exponentiated Gradient Reduction method (Agarwal et al. (2018)), which aims to solve a series of cost-sensitive prediction problems and return the predictor with the lowest error using fair regression constraints. We will use the pipeline method once again.

In [9]:
# initialize model and mitigation technique
model = LinearRegression()
inprocessing_model = ExponentiatedGradientReduction(constraints="BoundedGroupLoss", 
                                         loss='Square', min_val=-0.1, max_val=1.3, upper_bound=0.001,
                                         ).transform_estimator(model)

pipeline = Pipeline(
    steps=[
        ('scalar', StandardScaler()),
        ("bm_inprocessing", inprocessing_model),
    ]
)

X, y, group_a, group_b = train_data
fit_params = {
    "bm__group_a": group_a, 
    "bm__group_b": group_b
}

pipeline.fit(X, y, **fit_params)

X, y, group_a, group_b = test_data
predict_params = {
    "bm__group_a": group_a,
    "bm__group_b": group_b,
}
y_pred = pipeline.predict(X, **predict_params)
df = regression_bias_metrics(
    group_a,
    group_b,
    y_pred,
    y,
    metric_type='both'
)
y_exp_grad  = y_pred.copy()
df_exp_grad =df.copy()
df_exp_grad

,Value,Reference
Metric,,
Disparate Impact Q90,0.069697,1
Disparate Impact Q80,0.125455,1
Disparate Impact Q50,0.434266,1
Statistical Parity Q50,-0.633729,0
No Disparate Impact Level,-0.930456,-
Average Score Difference,-1.463753,0
Z Score Difference,-1.891108,0
Max Statistical Parity,0.643874,0
Statistical Parity AUC,0.395300,0


#### 3.3 - Post-Processing Methods for Bias Mitigation

The final level of bias mitigation we will be looking at is post-processing. Post-processing is applied after training to the outputs of the model. The original data nor the model are modified in any way. The post-processing technique you will be implementing is the Wasserstein Barycenters (Chzhen, Evgenii, et al. 2020). The algorithm aims to find a fair regressor with a demographic parity constraint under the assumption that the sensitive attribute is avail for prediction.

<font color='red'> **Task 2**
- **Implement the Wasserstein Barycenter post-processing technique (WassersteinBarycenter) using the pipeline method. Generate a summary of bias metrics and use the summary to comment on the performance of the algorithm. (Hint: the order of pipeline steps is different than in the pre-processing example.)**
<font >

In [10]:
# model
model = LinearRegression()

# TODO add ("bm_postprocessing", WasserteinBarycenter()) to following pipeline
pipeline = Pipeline(
    steps=[
        ('scalar', StandardScaler()),
        ("model", model),
    ]
)

# train data
X, y, group_a, group_b = train_data
fit_params = {
    "bm__group_a": group_a, 
    "bm__group_b": group_b
}

# fitting
pipeline.fit(X, y, **fit_params)

# test data
X, y, group_a, group_b = test_data
predict_params = {
    "bm__group_a": group_a,
    "bm__group_b": group_b,
}

# predicting
y_pred = pipeline.predict(X, **predict_params)

# metrics
df = regression_bias_metrics(
    group_a,
    group_b,
    y_pred,
    y,
    metric_type='both'
)
y_wb  = y_pred.copy()
df_wb =df.copy()
df_wb

,Value,Reference
Metric,,
Disparate Impact Q90,0.985714,1
Disparate Impact Q80,0.906061,1
Disparate Impact Q50,0.863170,1
Statistical Parity Q50,-0.020422,0
No Disparate Impact Level,1.021537,-
Average Score Difference,-0.063446,0
Z Score Difference,-0.103833,0
Max Statistical Parity,0.112516,0
Statistical Parity AUC,0.042851,0


You should get the following results:

| Metric | Value | Reference |
| ---    | ---   | --- |
Disparate Impact Q90	|0.985714	|1|
Disparate Impact Q80	|0.906061	|1|
Disparate Impact Q50	|0.863170	|1|
Statistical Parity Q50	|-0.020422	|0|
No Disparate Impact Level	|1.021537	|-|
Average Score Difference	|-0.063446	|0|
Z Score Difference	|-0.103833	|0|
Max Statistical Parity	|0.112516	|0|
Statistical Parity AUC	|0.042851	|0|
RMSE Ratio	|0.391934	|1|
RMSE Ratio Q80	|0.428850	|1|
MAE Ratio	|0.360770	|1|
MAE Ratio Q80	|0.418453	|1|
Correlation Difference	|-0.023593	|0|
